In [8]:
import folium
import pandas as pd 

m = folium.Map(
    location=(24.161259, 120.631192
),
    zoom_start=13,
    tiles="Cartodb dark_matter"
    
)

folium.Marker(
    location=(24.161259, 120.631192),
    popup='site',
    tooltip='site'
).add_to(m)


In [9]:
import pandas as pd
from pathlib import Path

p = Path('data/merged_data/MRT.json')
print('Path ->', p.resolve())
print('Exists ->', p.exists())

if p.exists():
    try:
        df_mrtroute = pd.read_json(p)
    except ValueError:
        df_mrtroute = pd.read_json(p, lines=True)
    print('Loaded rows,cols ->', df_mrtroute.shape)
    display(df_mrtroute.head())
else:
    raise FileNotFoundError(f'MRT.json not found at: {p}')


Path -> C:\DCCG_Final_Project\site-analysis\data\merged_data\MRT.json
Exists -> True
Loaded rows,cols -> (18, 3)


,欄位名稱,數值,經緯度
0,北屯總站,21818,"24.189132,120.70864"
1,舊社,30414,"24.182289,120.707294"
2,松竹,80013,"24.180807,120.70145"
3,四維國小,53208,"24.171243,120.693295"
4,文心崇德,91029,"24.172194,120.684869"


In [12]:
# 將 MRT 站點加入地圖，並根據「數值」調整點的大小
import pandas as pd
import numpy as np

# 確認 df_mrtroute 存在
if 'df_mrtroute' not in globals():
    raise NameError('df_mrtroute 未定義，請先執行載入 MRT 的 cell')

# 解析經緯度欄位 (格式: 'lat,lon')
coords = df_mrtroute['經緯度'].astype(str).str.split(',', expand=True)
df_mrtroute['lat'] = coords[0].astype(float)
df_mrtroute['lon'] = coords[1].astype(float)

# 取出數值欄並轉為數字
vals = pd.to_numeric(df_mrtroute['數值'], errors='coerce').fillna(0)
min_val, max_val = vals.min(), vals.max()

# 定義映射函數 -> 半徑 pixels
min_radius, max_radius = 4, 18

def scale_radius(v):
    if pd.isna(v):
        v = 0
    if max_val == min_val:
        return (min_radius + max_radius) / 2
    return min_radius + (float(v) - min_val) / (max_val - min_val) * (max_radius - min_radius)

for _, r in df_mrtroute.iterrows():
    radius = scale_radius(r['數值'])
    folium.CircleMarker(
        location=(r['lat'], r['lon']),
        radius=radius,
        color='lightgreen',
        fill=True,
        fill_color='lightgreen',
        fill_opacity=0.6,
        popup=f"{r['欄位名稱']}<br>數值: {r['數值']}",
        tooltip=r['欄位名稱']
    ).add_to(m)

# 加上圖層控制，顯示並儲存
folium.LayerControl().add_to(m)

m




In [15]:
import pandas as pd
import numpy as np
import folium
import requests
from math import radians, cos, sin, asin, sqrt

# 1. 設定基地座標與過濾範圍
BASE_LAT, BASE_LON = 24.161259, 120.631192
RADIUS_KM = 0.5 

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dLat, dLon = radians(lat2 - lat1), radians(lon2 - lon1)
    a = sin(dLat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dLon / 2)**2
    return 2 * R * asin(sqrt(a))

def get_route_path(coords):
    """
    透過 OSRM API 取得沿道路的點座標
    coords: [(lat1, lon1), (lat2, lon2), ...]
    """
    # 將座標轉為 OSRM 格式: lon,lat;lon,lat
    loc_string = ";".join([f"{lon},{lat}" for lat, lon in coords])
    url = f"http://router.project-osrm.org/route/v1/driving/{loc_string}?overview=full&geometries=geojson"
    
    try:
        r = requests.get(url, timeout=5)
        res = r.json()
        if res.get('code') == 'Ok':
            # 回傳的是 [lon, lat]，要轉回 [lat, lon] 給 folium
            route_geometry = res['routes'][0]['geometry']['coordinates']
            return [[lat, lon] for lon, lat in route_geometry]
    except Exception as e:
        print(f"Routing error: {e}")
    return coords # 失敗時回傳原始點對點

# --- 初始化地圖 (修改為 dark_matter) ---
m = folium.Map(location=[BASE_LAT, BASE_LON], zoom_start=14, tiles="Cartodb dark_matter")

# 2. 繪製黃色虛線圓
folium.Circle(
    location=[BASE_LAT, BASE_LON],
    radius=500,
    color='yellow',
    weight=2,
    fill=True,
    fill_opacity=0.05,
    dash_array='10, 10', # 虛線設定
    tooltip="基地 1km 直徑範圍"
).add_to(m)

# --- 篩選與繪圖邏輯 ---
def check_route_in_range(group):
    for _, row in group.iterrows():
        if pd.notna(row['Lat']) and pd.notna(row['Lon']):
            if haversine(BASE_LAT, BASE_LON, row['Lat'], row['Lon']) <= RADIUS_KM:
                return True
    return False

group_cols = ['RouteDisplay', 'Direction']
pink_palette = ['#FFC0CB', '#FFB6C1', '#FF69B4', '#FF1493', '#F48FB1', '#F06292', '#F8BBD0']

# 為了加快速度，限制處理的路線數量（可視情況刪除此限制）
filtered_groups = [g for _, g in df_itcroute.groupby(group_cols, sort=False) if check_route_in_range(g)]

print(f"正在優化 {len(filtered_groups)} 條路線的路網連線...")

for g in filtered_groups:
    g_sorted = g.sort_values('StopSequence')
    route_name = g_sorted['RouteDisplay'].iloc[0]
    direction = g_sorted['Direction'].iloc[0]
    color = pink_palette[hash(str(route_name)) % len(pink_palette)]
    
    # 取得站點座標
    stop_coords = list(zip(g_sorted['Lat'].astype(float), g_sorted['Lon'].astype(float)))
    
    if len(stop_coords) >= 2:
        # 3. 呼叫 OSRM 取得沿路徑的座標
        road_path = get_route_path(stop_coords)
        
        folium.PolyLine(
            locations=road_path, 
            color=color, 
            weight=4, # 調整粗細
            opacity=0.8, 
            popup=f"{route_name} ({'去' if direction==0 else '回'})"
        ).add_to(m)
        
    # 標示站點
    for _, r in g_sorted.iterrows():
        folium.CircleMarker(
            location=(r['Lat'], r['Lon']),
            radius=3,
            color='white', # 站點改用白色在深底圖更亮
            fill=True,
            fill_color=color,
            fill_opacity=1,
            tooltip=r.get('StopName')
        ).add_to(m)

m.save('pro_road_map.html')

正在優化 20 條路線的路網連線...
